In [ ]:
import os
from glob import glob

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
from zipfile import ZipFile

%matplotlib inline

In [ ]:
# Hyperparameter Tunning
num_epochs = 3
batch_size = 32
num_classes = 2
input_shape = (32, 32, 3)

In [ ]:
with ZipFile('../input/aerial-cactus-identification/train.zip', 'r') as zipObj:
    zipObj.extractall()

train_df = pd.read_csv('../input/aerial-cactus-identification/train.csv', dtype = str)                                                 

In [ ]:
with ZipFile('../input/aerial-cactus-identification/test.zip', 'r') as zipObj:
    zipObj.extractall()

test_df = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv', dtype = str)

In [ ]:
train_dir = np.array(glob('../working/train/'))
test_dir = np.array(glob('../working/test/'))

In [ ]:
id_value = train_df['id'].values
id_value_test = test_df['id'].values
id_value

In [ ]:
train_path = train_dir + id_value
test_path = test_dir + id_value_test
train_path[:20]

In [ ]:
train_path_2 = np.array(glob('../working/train/*.jpg'), dtype = object)
train_path_2[:20]

In [ ]:
class_list = ['0', '1']
classes = np.array(train_df['has_cactus']).reshape([len(train_df['has_cactus']),-1])
onehot_classes = np.array(np.array(classes) == class_list, dtype = np.uint8)
onehot_classes

In [ ]:
def get_onehot(path):
    cls_name = np.unique(train_df['has_cactus'])
    return tf.cast(tf.constant(classes) == cls_name, dtype=tf.uint8)

def read_image(path):
    image_raw = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image_raw)
    return image

def get_data(path):
    image = read_image(path)
    onehot = get_onehot(path)
    return image, onehot

def get_data_onehot(path, onehot):
    image = read_image(path)
    return image, onehot

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((train_path, onehot_classes))
dataset = dataset.map(get_data_onehot)
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(100)
dataset = dataset.repeat()

#path, onehot = next(iter(dataset))

In [ ]:
inputs = layers.Input(input_shape)
#Feature Extraction
net = layers.Conv2D(32, (3, 3), padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D((2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(64, (3, 3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D((2, 2))(net)
net = layers.Dropout(0.25)(net)

#Fully connected layers
net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.25)(net)
net = layers.Dense(2)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net, name = 'Basic_CNN')

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(dataset,
          epochs = 5,
          steps_per_epoch=len(train_path)//batch_size
         )

In [ ]:
class_list = ['0', '1']
classes = np.array(test_df['has_cactus']).reshape([len(test_df['has_cactus']),-1])
onehot_classes = np.array(np.array(classes) == class_list, dtype = np.uint8)
onehot_classes

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_path))
test_dataset = test_dataset.map(read_image)
test_dataset = test_dataset.batch(batch_size)

In [ ]:
preds = model.predict(test_dataset)
pred_result = np.argmax(preds, -1)

In [ ]:
test_df['has_cactus'] = pred_result
test_df.to_csv('../working/submission.csv', index = False)

In [ ]:
import shutil
shutil.rmtree("train")
shutil.rmtree("test")